In [16]:
%matplotlib inline
import os
import time
import warnings
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import ulmo
import requests

warnings.filterwarnings("ignore")
plt.rcParams['figure.figsize'] = [10, 8]

# =============================================================================
# 1. LOAD DOMAIN AND GET SNOTEL SITES (your existing code)
# =============================================================================

gpkg_file_path = '/Users/wcurrier/Documents/Data/UCRB_NRCS_Forecast_Points_Shape_2.gpkg'
df = gpd.read_file(gpkg_file_path)
df = df.to_crs(epsg=4326)

# Import state boundaries
states_url = 'http://eric.clst.org/assets/wiki/uploads/Stuff/gz_2010_us_040_00_5m.json'
states_gdf = gpd.read_file(states_url)

# Get SNOTEL sites via CUAHSI WOF
wsdlurl = 'http://hydroportal.cuahsi.org/Snotel/cuahsi_1_1.asmx?WSDL'
sites = ulmo.cuahsi.wof.get_sites(wsdlurl)

sites_df = pd.DataFrame.from_dict(sites, orient='index')
sites_df = sites_df.dropna()
sites_df['geometry'] = [Point(float(loc['longitude']), float(loc['latitude'])) for loc in sites_df['location']]
sites_df = sites_df.drop(columns='location')
sites_df = sites_df.astype({"elevation_m": float})
sites_gdf = gpd.GeoDataFrame(sites_df, geometry='geometry')
sites_gdf.crs = 'EPSG:4326'

# Clip to domain
poly = df.geometry.unary_union
sites_gdf_crb = sites_gdf[sites_gdf.geometry.intersects(poly)].copy()
sites_gdf_crb = sites_gdf_crb.assign(siteStr=sites_gdf_crb.index.str[:])

print(f"Found {len(sites_gdf_crb)} SNOTEL sites within the domain.")

# =============================================================================
# 2. CONVERT CUAHSI SITE CODES TO NRCS TRIPLETS
# =============================================================================
# CUAHSI WOF codes look like: "SNOTEL:622_CO_SNTL"
# NRCS triplets look like:    "622:CO:SNTL"

def cuahsi_to_triplet(cuahsi_code):
    """Convert a CUAHSI WOF site code to an NRCS station triplet."""
    # Strip the 'SNOTEL:' prefix if present
    code = cuahsi_code.split(":")[-1] if ":" in cuahsi_code else cuahsi_code
    # Replace underscores with colons
    parts = code.split("_")
    if len(parts) >= 3:
        # Format: siteID_STATE_NETWORK  ->  siteID:STATE:NETWORK
        return f"{parts[0]}:{parts[1]}:{parts[2]}"
    return None

sites_gdf_crb["triplet"] = sites_gdf_crb.index.map(cuahsi_to_triplet)
sites_gdf_crb = sites_gdf_crb.dropna(subset=["triplet"])

print(f"{len(sites_gdf_crb)} sites with valid triplet codes.")
print("Example triplets:", sites_gdf_crb["triplet"].head().tolist())



In [ ]:
# Save sites_gdf_crb so we can skip Steps 1-2 next time
sites_gdf_crb.to_file("/Users/wcurrier/Documents/Data/sites_gdf_crb.gpkg", driver="GPKG")
print("Saved sites_gdf_crb.gpkg")

In [11]:
# =============================================================================
# 3. FETCH SMS AND STO FOR ALL SITES - PRODUCTION RUN
# =============================================================================

import os
import time
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import pandas as pd
import geopandas as gpd
import io
from datetime import datetime

# -- Reload sites_gdf_crb --
sites_gdf_crb = gpd.read_file("/Users/wcurrier/Documents/Data/sites_gdf_crb.gpkg")

REPORT_GEN_BASE = "https://wcc.sc.egov.usda.gov/reportGenerator/view_csv"
BEGIN_DATE = "1999-01-01"
END_DATE = datetime.now().strftime("%Y-%m-%d")
date_index = pd.date_range(start=BEGIN_DATE, end=END_DATE, freq="D")

DEPTHS = [-2, -4, -8, -20, -40]
ELEMENT_STRINGS = [f"SMS:{d}:value" for d in DEPTHS] + [f"STO:{d}:value" for d in DEPTHS]
ELEMENTS_URL_PART = ",".join(ELEMENT_STRINGS)

# Session with retries
session = requests.Session()
retries = Retry(total=3, backoff_factor=2, status_forcelist=[500, 502, 503, 504])
session.mount("https://", HTTPAdapter(max_retries=retries))


def fetch_soil_data_for_site(triplet):
    """Fetch SMS/STO at all depths, return DataFrame or None."""
    url = (f"{REPORT_GEN_BASE}/customSingleStationReport/daily/"
           f"{triplet}/{BEGIN_DATE},{END_DATE}/{ELEMENTS_URL_PART}")
    try:
        resp = session.get(url, timeout=120)
        resp.raise_for_status()
    except Exception as e:
        return None, str(e)

    lines = resp.text.split("\n")
    data_lines = [l for l in lines if not l.startswith("#") and l.strip()]
    if len(data_lines) < 2:
        return None, "no data rows"

    try:
        df = pd.read_csv(io.StringIO("\n".join(data_lines)))
    except Exception as e:
        return None, f"parse error: {e}"

    if df.empty:
        return None, "empty CSV"

    # Parse dates
    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df = df.dropna(subset=[date_col]).set_index(date_col)
    df.index.name = "date"

    # Rename columns
    rename_map = {}
    for col in df.columns:
        cl = col.lower()
        new = col
        for depth in DEPTHS:
            ds = f"{abs(depth)}in"
            if f"-{ds}" in cl or f" {ds}" in cl:
                if "soil moisture" in cl:
                    new = f"SMS_{depth}"
                elif "soil temperature" in cl:
                    new = f"STO_{depth}"
                break
        rename_map[col] = new
    df = df.rename(columns=rename_map)

    # Convert to numeric, drop all-NaN columns
    for col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    df = df.dropna(axis=1, how="all")

    if df.empty or df.shape[1] == 0:
        return None, "all columns NaN"

    # Reindex to full date range
    df = df.reindex(date_index)
    df.index.name = "date"
    df = df.dropna(axis=1, how="all")

    return df, None


# =============================================================================
# RUN ALL SITES
# =============================================================================
soil_data = {}
sites_with_data = []
sites_no_data = []
n_sites = len(sites_gdf_crb)

print(f"Fetching soil data for {n_sites} sites...")
print(f"Date range: {BEGIN_DATE} to {END_DATE}")
print("=" * 60)

t_start = time.time()

for i, (idx, row) in enumerate(sites_gdf_crb.iterrows()):
    triplet = row["triplet"]
    name = row.get("name", triplet)

    site_df, err = fetch_soil_data_for_site(triplet)

    if site_df is not None:
        soil_data[triplet] = site_df
        n_sms = sum(1 for c in site_df.columns if c.startswith("SMS"))
        n_sto = sum(1 for c in site_df.columns if c.startswith("STO"))
        first = site_df.first_valid_index()
        last = site_df.last_valid_index()
        sites_with_data.append({
            "triplet": triplet, "name": name,
            "n_sms": n_sms, "n_sto": n_sto,
            "first_valid": first, "last_valid": last,
        })
        print(f"  [{i+1}/{n_sites}] {name}: {n_sms} SMS, {n_sto} STO "
              f"({first.date()} to {last.date()})")
    else:
        sites_no_data.append({"triplet": triplet, "name": name, "reason": err})
        print(f"  [{i+1}/{n_sites}] {name}: no soil data")

    time.sleep(1.0)

elapsed = time.time() - t_start

print("\n" + "=" * 60)
print(f"DONE in {elapsed/60:.1f} min")
print(f"  Sites with data: {len(soil_data)}/{n_sites}")
print(f"  Sites without:   {len(sites_no_data)}/{n_sites}")
print("=" * 60)

# =============================================================================
# SAVE RESULTS
# =============================================================================
output_dir = "/Users/wcurrier/Documents/Data/snotel_soil/"
os.makedirs(output_dir, exist_ok=True)

# Save individual site CSVs
for triplet, sdf in soil_data.items():
    safe_name = triplet.replace(":", "_")
    sdf.to_csv(os.path.join(output_dir, f"{safe_name}_soil.csv"))

# Save summary
if sites_with_data:
    summary_df = pd.DataFrame(sites_with_data)
    summary_df.to_csv(os.path.join(output_dir, "sites_with_data_summary.csv"), index=False)

if sites_no_data:
    no_data_df = pd.DataFrame(sites_no_data)
    no_data_df.to_csv(os.path.join(output_dir, "sites_no_data.csv"), index=False)

# Save combined multi-index DataFrame
if soil_data:
    frames = []
    for triplet, sdf in soil_data.items():
        sdf_copy = sdf.copy()
        sdf_copy["triplet"] = triplet
        frames.append(sdf_copy.reset_index())
    combined_df = pd.concat(frames, ignore_index=True)
    combined_df = combined_df.set_index(["triplet", "date"]).sort_index()
    combined_df.to_csv(os.path.join(output_dir, "all_sites_soil_combined.csv"))
    print(f"\nCombined DataFrame: {combined_df.shape}")

print(f"Saved to {output_dir}")

Fetching soil data for 139 sites...
Date range: 1999-01-01 to 2026-02-26
  [1/139] Alta Lakes: 3 SMS, 3 STO (2025-09-26 to 2026-02-26)
  [2/139] Arapaho Ridge: 3 SMS, 3 STO (2003-08-01 to 2026-02-26)
  [3/139] Arrow: no soil data
  [4/139] Battle Mountain: no soil data
  [5/139] Bear River: no soil data
  [6/139] Beaver Ck Village: no soil data
  [7/139] Beaver Spring: no soil data
  [8/139] Berthoud Summit: 4 SMS, 4 STO (2002-07-05 to 2026-02-26)
  [9/139] Big Sandy Opening: 3 SMS, 3 STO (2001-07-13 to 2025-04-11)
  [10/139] Bison Lake: no soil data
  [11/139] Black Flat-U.M. Ck: 3 SMS, 3 STO (2004-06-26 to 2026-02-26)
  [12/139] Black Mesa: 4 SMS, 4 STO (2012-08-24 to 2026-02-26)
  [13/139] Blacks Fork Jct: 3 SMS, 3 STO (2010-10-23 to 2026-02-26)
  [14/139] Blind Bull Sum: no soil data
  [15/139] Brown Duck: 3 SMS, 3 STO (2003-08-07 to 2026-02-26)
  [16/139] Buck Flat: 3 SMS, 3 STO (2002-11-02 to 2026-02-26)
  [17/139] Buck Pasture: 3 SMS, 3 STO (2012-09-12 to 2026-02-26)
  [18/139] 

In [15]:
summary_df

,triplet,name,n_sms,n_sto,first_valid,last_valid
0,1344:CO:SNTL,Alta Lakes,3,3,2025-09-26,2026-02-26
1,1030:CO:SNTL,Arapaho Ridge,3,3,2003-08-01,2026-02-26
2,335:CO:SNTL,Berthoud Summit,4,4,2002-07-05,2026-02-26
3,342:WY:SNTL,Big Sandy Opening,3,3,2001-07-13,2025-04-11
4,348:UT:SNTL,Black Flat-U.M. Ck,3,3,2004-06-26,2026-02-26
...,...,...,...,...,...,...
84,1141:CO:SNTL,Upper Taylor,3,3,2009-10-10,2026-02-26
85,864:UT:SNTL,White River #1,3,3,2003-05-30,2026-02-26
86,543:UT:SNTL,Wilbur Bench,3,3,2004-07-24,2026-02-26
87,874:CO:SNTL,Wolf Creek Summit,3,3,2003-08-08,2026-02-26


In [14]:
combined_df

SMS_-2  SMS_-8  SMS_-20  STO_-2  STO_-8  STO_-20  \
triplet      date                                                           
1014:CO:SNTL 1999-01-01     NaN     NaN      NaN     NaN     NaN      NaN   
             1999-01-02     NaN     NaN      NaN     NaN     NaN      NaN   
             1999-01-03     NaN     NaN      NaN     NaN     NaN      NaN   
             1999-01-04     NaN     NaN      NaN     NaN     NaN      NaN   
             1999-01-05     NaN     NaN      NaN     NaN     NaN      NaN   
...                         ...     ...      ...     ...     ...      ...   
970:CO:SNTL  2026-02-22     NaN     6.6      9.2     NaN    33.0     33.0   
             2026-02-23     NaN     6.5      9.1     NaN    32.0     33.0   
             2026-02-24     NaN     6.2      8.8     NaN    32.0     33.0   
             2026-02-25     NaN     6.3      8.5     NaN    32.0     33.0   
             2026-02-26     NaN     6.1      8.4     NaN    32.0     33.0   

                         SMS_-4  STO_-4  SMS_-40  STO_-40  
triplet      date                                          
1014:CO:SNTL 1999-01-01     NaN     NaN      NaN      NaN  
             1999-01-02     NaN     NaN      NaN      NaN  
             1999-01-03     NaN     NaN      NaN      NaN  
             1999-01-04     NaN     NaN      NaN      NaN  
             1999-01-05     NaN     NaN      NaN      NaN  
...                         ...     ...      ...      ...  
970:CO:SNTL  2026-02-22     NaN     NaN      6.0     33.0  
             2026-02-23     NaN     NaN      6.0     33.0  
             2026-02-24     NaN     NaN      5.9     33.0  
             2026-02-25     NaN     NaN      5.9     33.0  
             2026-02-26     NaN     NaN      5.9     33.0  

[882791 rows x 10 columns]